# Linear regression

In [6]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from itertools import product

import numpy as np
import xarray as xr
import xarray.ufuncs as xf
import matplotlib.pyplot as plt
from scipy.optimize import lsq_linear

from system_identification.lsqr_model import LeastSquaresModel

In [8]:
data = xr.open_dataset("data_smoothed.nc")
data

<xarray.Dataset>
Dimensions:                 (dim_0: 4, dim_1: 4, t: 10001, x_idx: 4)
Coordinates:
  * t                       (t) float64 0.0 0.01 0.02 0.03 ... 99.98 99.99 100.0
Dimensions without coordinates: dim_0, dim_1, x_idx
Data variables: (12/35)
    p_k1k1                  (t, dim_0, dim_1) float64 ...
    p_kk1                   (t, dim_0, dim_1) float64 ...
    phi                     (t, dim_0, dim_1) float64 ...
    gamma                   (t, dim_0, dim_1) float64 ...
    ps                      (t, dim_0, dim_1) float64 ...
    x_k1k1                  (t, x_idx) float64 ...
    ...                      ...
    w_smoothed_std          (t) float64 ...
    C_alpha_u_smoothed      (t) float64 ...
    C_alpha_u_smoothed_std  (t) float64 ...
    vm_filtered             (t) float64 ...
    vm_smoothed             (t) float64 ...
    alpha_estimate          (t) float64 ...

In [11]:
model = LeastSquaresModel.new_polynomial(
    n_inputs=2,
    range=[[-np.inf, np.inf], [-np.inf, np.inf]],
    order=20)
model

In [5]:
inputs = np.vstack(
    (data.alpha_m.values,
     data.beta_m.values)).T[..., None]
reference_outputs = data.c_m.values.reshape((-1, 1, 1))

In [6]:
model.train(inputs, reference_outputs)

In [7]:
evaluated = model.evaluate(inputs)

In [8]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(data.alpha_m, data.beta_m, data.c_m, s=0.1)
ax.scatter(data.alpha_m, data.beta_m, evaluated[:, 0, 0], s=0.1)
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …